### Deploy and Interact with LUCE contract
The aim of this notebook is to compile, deploy and interact with all functions of the LUCE smart contract (v3).

### Compile contract from file

In [72]:
# Import libraries
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [73]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [74]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [75]:
#compiled_sol

In [76]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [77]:
#contract_interface

In [78]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

In [79]:
#contract_interface['bin']

### Deploy

In [80]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [85]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

AttributeError: module 'web3.eth' has no attribute 'accounts'

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [63]:
# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

In [64]:
# Submit the transaction that deploys the contract
tx_hash = Luce.constructor().transact()

ValueError: {'code': -32601, 'message': 'The method eth_sendTransaction does not exist/is not available'}

### Obtain Transcation Receipt

In [ ]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [65]:
# Obtain address of freshly deployed contract
tx_receipt.contractAddress

'0x6ADA382390267D20Fed478069768FCd39FcBEd23'

### Interact with contract

In [66]:
# Create python instance of deployed contract
luce1 = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

**publishData**  
```solidity
publishData(string memory _newdescription, string memory _link, uint _licence)
```

In [67]:
# Publish Data
tx_hash = luce1.functions.publishData("example description", "my link", 6).transact()

ValueError: {'code': -32601, 'message': 'The method eth_sendTransaction does not exist/is not available'}

**setLicence**
```solidity
setLicence(uint newLicence)
```

In [ ]:
# Set Licence
_ = luce1.functions.setLicence(5).transact()

From here on I am using `_` to suppress the output of the transaction hash. I do this since transactions are mined instantaneously in the local ganache test setting. On the testnet and mainnet we would store the transaction hash, then wait until a transaction receipt is available and only then we can be sure that the transaction went through.

**getLicence**
```solidity
getLicence() public view returns(uint)
```

In [68]:
# Get Licence
luce1.functions.getLicence().call()

BadFunctionCallOutput: Could not transact with/call contract function, is contract deployed correctly and chain synced?

**updateData**
```solidity
updateData(string memory updateDescr, string memory _newlink)
```

In [ ]:
# Update Data
_ = luce1.functions.updateData("New Description", "New Link").transact()

**getLink**
```solidity
getLink(uint token) public view returns(string memory)
```

In [69]:
luce1.functions.getLink(1).call()

BadFunctionCallOutput: Could not transact with/call contract function, is contract deployed correctly and chain synced?

All LUCE functions up to this point are working.

---

We can also query the dataProvider and dataDescription since public variables in smart contracts automatically have a getter method. We simply call them by their name to read the current variable state:

**dataProvider**

In [70]:
luce1.functions.dataProvider().call()

BadFunctionCallOutput: Could not transact with/call contract function, is contract deployed correctly and chain synced?

**dataDescription**

In [ ]:
luce1.functions.dataDescription().call()

---

**UPDATE:**  
All working as expected now

**addDataRequester**
```solidity
addDataRequester(uint purposeCode, uint licenceType) public returns(uint)
```

In [71]:
luce1.functions.addDataRequester(1,5).transact()

ValueError: {'code': -32601, 'message': 'The method eth_sendTransaction does not exist/is not available'}

**renewToken**
```solidity
renewToken(uint compliance) public returns(uint token)
```

In [ ]:
luce1.functions.renewToken(2).transact()
# Error: Need to agree on licence first